<a href="https://colab.research.google.com/github/yangliupku/cs336_assignment2_systems/blob/main/notebooks/FlashAttention2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from einops import einsum
import math

In [53]:
B = 64
D = 1024
Bq = 10
Bk = 10

In [15]:
Q = torch.rand(B, D)
K = torch.rand(B, D)
V = torch.rand(B, D)

In [36]:
S = einsum(Q, K, 'bq d, bk d -> bq bk') / math.sqrt(D)
P = torch.softmax(S, dim=-1)
O = einsum(P, V, 'bq bk, bk d -> bq d')
L = torch.log(torch.sum(torch.exp(S), axis=-1))
print(O)
print(O.shape)
print(L)
print(L.shape)

tensor([[0.4194, 0.4443, 0.5016,  ..., 0.5111, 0.4965, 0.5560],
        [0.4219, 0.4460, 0.5002,  ..., 0.5116, 0.4997, 0.5484],
        [0.4134, 0.4474, 0.4975,  ..., 0.5061, 0.4919, 0.5519],
        ...,
        [0.4248, 0.4435, 0.4978,  ..., 0.5070, 0.4936, 0.5448],
        [0.4246, 0.4458, 0.4913,  ..., 0.5067, 0.4902, 0.5507],
        [0.4237, 0.4470, 0.4944,  ..., 0.5060, 0.4925, 0.5556]])
torch.Size([64, 1024])
tensor([12.0023, 12.2469, 11.9829, 11.8613, 12.1185, 12.3269, 12.2075, 11.8744,
        12.1786, 12.1615, 12.3319, 12.2788, 11.8585, 12.2465, 12.0639, 11.9156,
        12.0999, 12.0783, 12.0229, 12.3031, 11.8806, 12.2182, 12.1552, 12.3759,
        12.1924, 12.1719, 12.2220, 12.0073, 12.4595, 12.0897, 12.0852, 12.2045,
        12.1742, 11.9323, 12.4415, 12.2295, 12.0696, 11.9311, 12.2001, 12.2567,
        11.8395, 12.1948, 12.1401, 12.1884, 12.1386, 12.2132, 12.1860, 12.0523,
        12.1982, 12.2127, 11.8483, 11.9917, 12.3106, 12.3414, 11.9267, 12.0404,
        12.2204, 12

In [54]:
Tq = math.ceil(B/Bq)
Tk = math.ceil(B/Bk)

In [63]:
OO = []
LL = []
for i in range(Tq):
  Oi = torch.zeros(Bq, D)
  li = torch.zeros(Bq,)
  mi = torch.ones(Bq,)*torch.tensor(float('-inf'))
  for j in range(Tk):
    Qi = Q[i*Bq:(i+1)*Bq, :]
    Kj = K[j*Bk:(j+1)*Bk, :]
    Vj = V[j*Bk:(j+1)*Bk, :]
    # pad 0 to Qi
    if Qi.shape[0] < Bq:
      Qi = torch.cat([Qi, torch.zeros(Bq-Qi.shape[0], D)], axis=0)
    # pad 0 to Kj, Vj
    if Kj.shape[0] < Bk:
      Kj = torch.cat([Kj, torch.zeros(Bk-Kj.shape[0], D)], axis=0)
    if Vj.shape[0] < Bk:
      Vj = torch.cat([Vj, torch.zeros(Bk-Vj.shape[0], D)], axis=0)
    prev_mi = mi
    Sij = einsum(Qi, Kj, 'q d, k d -> q k') / math.sqrt(D)
    mi = torch.maximum(prev_mi, torch.max(Sij, axis=-1).values)
    Pij = torch.exp(Sij - mi.unsqueeze(-1))
    scaling_factor = torch.exp(prev_mi-mi)
    li = scaling_factor*li +torch.sum(Pij, axis=-1)
    Oi = scaling_factor.unsqueeze(-1)*Oi + Pij@Vj

  Oi = (1/li).unsqueeze(-1)*Oi
  li = mi + torch.log(li)
  OO.append(Oi)
  LL.append(li)

O = torch.concat(OO, axis=0)[:Q.shape[0],:]
L = torch.concat(LL, axis=0)[:Q.shape[0],]
print(O)
print(O.shape)
print(L)
print(L.shape)

tensor([[0.4194, 0.4443, 0.5015,  ..., 0.5111, 0.4965, 0.5559],
        [0.4219, 0.4459, 0.5002,  ..., 0.5116, 0.4997, 0.5484],
        [0.4134, 0.4474, 0.4975,  ..., 0.5061, 0.4918, 0.5518],
        ...,
        [0.4248, 0.4435, 0.4977,  ..., 0.5070, 0.4936, 0.5448],
        [0.4245, 0.4458, 0.4913,  ..., 0.5067, 0.4901, 0.5507],
        [0.4237, 0.4470, 0.4944,  ..., 0.5060, 0.4925, 0.5556]])
torch.Size([64, 1024])
tensor([12.0024, 12.2470, 11.9829, 11.8614, 12.1185, 12.3270, 12.2075, 11.8744,
        12.1786, 12.1615, 12.3319, 12.2788, 11.8585, 12.2465, 12.0639, 11.9156,
        12.0999, 12.0783, 12.0229, 12.3031, 11.8806, 12.2182, 12.1553, 12.3759,
        12.1925, 12.1719, 12.2221, 12.0073, 12.4596, 12.0897, 12.0852, 12.2046,
        12.1743, 11.9324, 12.4416, 12.2295, 12.0696, 11.9312, 12.2002, 12.2567,
        11.8395, 12.1948, 12.1402, 12.1884, 12.1386, 12.2132, 12.1861, 12.0523,
        12.1983, 12.2127, 11.8483, 11.9918, 12.3106, 12.3414, 11.9267, 12.0404,
        12.2205, 12

In [61]:
O.shape

torch.Size([64, 1024])

In [85]:
O.shape

torch.Size([64, 1024])

In [80]:
O.shape

torch.Size([64, 1024])

In [66]:
torch.concat([a,b], axis=0)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [22]:
a = [1,2,3]

In [24]:
a[0:8]

[1, 2, 3]